# 3. Train-Predict Mix3model

## Result:
- Kaggle score: 

## Tensorboard
- Input at command: tensorboard --logdir=./log
- Input at browser: http://127.0.0.1:6006

## Reference
- https://www.kaggle.com/codename007/a-very-extensive-landmark-exploratory-analysis

## Import PKGs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import gc
import time
import zipfile
import pickle
import math
import pdb
import h5py
from PIL import Image
import shutil

from tqdm import tqdm
import multiprocessing

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Run name

In [2]:
project_name = 'Google_LandMark_Rec'
step_name = '3. Train-Predict_Mix3model'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: Google_LandMark_Rec_3. Train-Predict_Mix3model_20180410_123455


## 项目文件夹

In [3]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t' + input_folder)
print('output_folder: \t\t\t' + output_folder)
print('model_folder: \t\t\t' + model_folder)
print('feature_folder: \t\t' + feature_folder)
print('post_pca_feature_folder: \t' + post_pca_feature_folder)
print('log_folder: \t\t\t' + log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

input_folder: 			/data1/kaggle/landmark-recognition-challenge/input
output_folder: 			/data1/kaggle/landmark-recognition-challenge/output
model_folder: 			/data1/kaggle/landmark-recognition-challenge/model
feature_folder: 		/data1/kaggle/landmark-recognition-challenge/feature
post_pca_feature_folder: 	/data1/kaggle/landmark-recognition-challenge/post_pca_feature
log_folder: 			/data1/kaggle/landmark-recognition-challenge/log


In [4]:
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## 获取landmark_id集合

In [5]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(10))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(10))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231
5,2c9c54b62f0a6a37,https://lh5.googleusercontent.com/-mEaSECO7D-4...,10400
6,0aac70a1de44ced6,http://lh6.ggpht.com/-cJMh9AYQGk8/SOkF_Q5PrjI/...,9779
7,de770bc720f8e714,https://lh4.googleusercontent.com/-Q_FvRlwaaa8...,11288
8,dc9457d703e612ad,https://lh3.googleusercontent.com/-Px33Q-wekRI...,13170
9,3060f5f75d936abb,http://lh3.ggpht.com/-KXyELwqwp_Q/Ry-qmQAqwUI/...,6051


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...
2,0001bbb682d45002,https://lh3.googleusercontent.com/-kloLenz1xZk...
3,0002362830cfe3a3,https://lh3.googleusercontent.com/-N6z79jNZYTg...
4,000270c9100de789,https://lh3.googleusercontent.com/-keriHaVOq1U...
5,0002b0fab5d3ccc4,https://lh3.googleusercontent.com/-ciWklpsrab8...
6,000396be3c24830a,https://lh3.googleusercontent.com/-6W9F179t59Q...
7,000506dc6ab3a40e,https://lh3.googleusercontent.com/-_XHsAXB2LZA...
8,0005292fc4b005a3,https://lh3.googleusercontent.com/-RBZ4F1ZKNc0...
9,0005456a82264bc8,https://lh3.googleusercontent.com/-MRK7_uiKO6A...


In [6]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']
unique_landmark_ids = list(set(train_landmark_id))
len_unique_landmark_ids = len(unique_landmark_ids)
print(unique_landmark_ids[:10]) # 确认landmark_id是从0开始
print('len(unique_landmark_ids)=%d' % len_unique_landmark_ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len(unique_landmark_ids)=14951


## 预览sample_submission.csv

In [7]:
sample_submission_csv = pd.read_csv(sample_submission_folder)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

sample_submission_csv.shape is (117703, 2).


,id,landmarks
0,000088da12d664db,8815 0.03
1,0001623c6d808702,7249 0.61


## 加载feature

In [8]:
def load_h5(file_name):
    print(file_name)
    with h5py.File(file_name, 'r') as h:
        x_train = np.array(np.array(h['train']))
        y_train = np.array(h['train_labels'])
#         x_val.append(np.array(h['val']))
#         y_val = np.array(h['val_labels'])
        x_test = np.array(np.array(h['test']))
        return x_train, y_train, x_test

In [9]:
%%time
model_name = 'InceptionV3'
time_str = '200_20180312-050926'
feature_Xception = os.path.join(feature_folder, 'feature_%s_%s.h5' % (model_name, time_str))

x_train, y_train, x_test = load_h5(feature_Xception)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

len_data = x_train.shape[0]
len_test = x_test.shape[0]
print('len_data: %s' % len_data)
print('len_test: %s' % len_test)

print(y_train[:10])

/data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionV3_200_20180312-050926.h5
(1219426, 2048)
(1219426,)
(115942, 2048)
len_data: 1219426
len_test: 115942
[0 0 0 0 0 0 0 0 0 0]
CPU times: user 5.91 s, sys: 19.1 s, total: 25 s
Wall time: 25 s


In [10]:
%%time
from sklearn.utils import shuffle
data_indeces = list(range(len_data))
print(data_indeces[:10])

data_indeces = shuffle(data_indeces)
print(data_indeces[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[600092, 1157496, 502486, 314777, 24106, 601449, 1142329, 877254, 506515, 750788]
CPU times: user 528 ms, sys: 32 ms, total: 560 ms
Wall time: 557 ms


In [11]:
from sklearn.model_selection import train_test_split
train_indeces, val_indeces = train_test_split(data_indeces[:30*10000], test_size=0.01, random_state=2018, shuffle=False)

print('len(train_indeces)=%s' % len(train_indeces))
print(train_indeces[:10])
print('len(val_indeces)=%s' % len(val_indeces))
print(val_indeces[:10])

len(train_indeces)=297000
[600092, 1157496, 502486, 314777, 24106, 601449, 1142329, 877254, 506515, 750788]
len(val_indeces)=3000
[122593, 80372, 90019, 1105459, 984448, 308365, 821931, 528256, 579445, 141403]


In [12]:
def load_h5_data(feature_folder, file_reg, model_name, time_str):
    x_data = {}
    y_data = {}
    
    feature_model = os.path.join(feature_folder, file_reg % (model_name, time_str))
    with h5py.File(feature_model, 'r') as h:
        x_data = np.array(h['train'])
        y_data = np.array(h['train_labels'])
    return x_data, y_data

def load_h5_test(feature_folder, file_reg, model_name, time_str):
    x_test = {}
    
    feature_model = os.path.join(feature_folder, file_reg % (model_name, time_str))
    with h5py.File(feature_model, 'r') as h:
        x_test = np.array(h['test'])
    return x_test

In [13]:
def is_files_existed(feature_folder, file_reg, model_names, time_strs):
    for model_name in model_names:
        for time_str in time_strs:
            file_name = file_reg % (model_name, time_str)
            file_path = os.path.join(feature_folder, file_name)
            if not os.path.exists(file_path):
                print('File not existed: %s' % file_path)
                return False
            else:
                print('File existed: %s' % file_path)
    return True

# Test
file_reg = 'feature_%s_%s.h5'
model_names = [
#     'MobileNet', 
#     'VGG16',
#     'VGG19',
#     'ResNet50',
#     'DenseNet121',
#     'DenseNet169',
#     'DenseNet201',
    'Xception', 
    'InceptionV3', 
    'InceptionResNetV2'
]
time_strs = [
    '200_20180312-050926', 
    '150_20180311-151108'
]

print(is_files_existed(feature_folder, file_reg, model_names, time_strs))


def time_str_generator(time_strs):
    while(1):
        for time_str in time_strs:
            print('  ' + time_str)
            yield time_str
            
# Test
time_str_gen = time_str_generator(time_strs)
for i in range(10):
    next(time_str_gen)

File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_Xception_200_20180312-050926.h5
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_Xception_150_20180311-151108.h5
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionV3_200_20180312-050926.h5
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionV3_150_20180311-151108.h5
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionResNetV2_200_20180312-050926.h5
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionResNetV2_150_20180311-151108.h5
True
  200_20180312-050926
  150_20180311-151108
  200_20180312-050926
  150_20180311-151108
  200_20180312-050926
  150_20180311-151108
  200_20180312-050926
  150_20180311-151108
  200_20180312-050926
  150_20180311-151108


In [14]:
%%time
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

def load_time_str_feature_data(data_indeces, feature_folder, file_reg, model_names, time_str):
    x_data_time_strs = []
    y_data_time_strs = None
    for model_name in model_names:
        x_data_time_str, y_data_time_str = load_h5_data(feature_folder, file_reg, model_name, time_str)
        x_data_time_str, y_data_time_str = x_data_time_str[data_indeces], y_data_time_str[data_indeces]
        # Around data to 3 decimals to calculate computation
#         x_data_time_str = np.round(x_data_time_str, decimals=3) # 导致后面的全链接神经网络分类器不收敛
        x_data_time_strs.append(x_data_time_str)
        y_data_time_strs = y_data_time_str
    x_data_time_strs = np.concatenate(x_data_time_strs, axis=-1)
#     print(x_data_time_strs.shape)
#     print(y_data_time_strs.shape)
    return x_data_time_strs, y_data_time_strs

def data_generator_folder(data_indeces, feature_folder, file_reg, model_names, time_strs, batch_size, num_classes):
    assert is_files_existed(feature_folder, file_reg, model_names, time_strs)

    time_str_gen = time_str_generator(time_strs)
    x_data, y_data = load_time_str_feature_data(data_indeces, feature_folder, file_reg, model_names, next(time_str_gen))
    len_x_data = len(x_data)
    start_index = 0
    end_index = 0
    while(1):
        end_index = start_index + batch_size
        if end_index < len_x_data:
#             print(start_index, end_index, end=' ')
            x_batch = x_data[start_index: end_index, :]
            y_batch = y_data[start_index: end_index]
            y_batch_cat = to_categorical(y_batch, num_classes)
            
            start_index = start_index + batch_size
#             print(x_batch.shape, y_batch_cat.shape)
            yield x_batch, y_batch_cat
        else:
            end_index = end_index-len_x_data
#             print(start_index, end_index, end=' ')
            x_data_old = np.array(x_data[start_index:, :], copy=True)
            y_data_old = np.array(y_data[start_index:], copy=True)
            # Load new datas
            x_data, y_data = load_time_str_feature_data(data_indeces, feature_folder, file_reg, model_names, next(time_str_gen))
#             x_data, y_data = shuffle(x_data, y_data, random_state=2018)
            len_x_data = len(x_data)
            gc.collect()
            x_batch = np.vstack((x_data_old, x_data[:end_index, :]))
            y_batch = np.concatenate([y_data_old, y_data[:end_index]])
            y_batch_cat = to_categorical(y_batch, num_classes)
            
            start_index = end_index
#             print(x_batch.shape, y_batch_cat.shape)
            yield x_batch, y_batch_cat
        
    
# x_train = np.concatenate([x_train_Xception, x_train_InceptionV3, x_train_InceptionResNetV2], axis=-1)

num_classes = len_unique_landmark_ids
print('num_classes: %s' % num_classes)

file_reg = 'feature_%s_%s.h5'
model_names = [
#     'MobileNet', 
#     'VGG16',
#     'VGG19',
#     'ResNet50',
#     'DenseNet121',
#     'DenseNet169',
#     'DenseNet201',
#     'Xception', 
#     'InceptionV3', 
    'InceptionResNetV2'
]
time_strs = [
    '200_20180312-050926', 
#     '150_20180311-151108'
]


batch_size = 8
print('*' * 60)
timesteps = len(model_names)
len_train_csv = len(train_indeces)
steps_per_epoch_train = int(len_train_csv/batch_size)
print('timesteps: %s' % timesteps)
print('len(train_data): %s' % len_train_csv)
print('batch_size: %s' % batch_size)
print('steps_per_epoch_train: %s' % steps_per_epoch_train)

train_gen = data_generator_folder(train_indeces, feature_folder, file_reg, model_names, time_strs, batch_size, num_classes)
batch_data = next(train_gen)
print(batch_data[0].shape, batch_data[1].shape)
batch_data = next(train_gen)
print(batch_data[0].shape, batch_data[1].shape)
# for i in range(steps_per_epoch_train*5):
#     next(train_gen)

print('*' * 60)
len_val_csv = len(val_indeces)
steps_per_epoch_val = int(len_val_csv/batch_size) + 1
print('len(val_data): %s' % len_val_csv)
print('batch_size: %s' % batch_size)
print('steps_per_epoch_val: %s' % steps_per_epoch_val)
val_gen = data_generator_folder(val_indeces, feature_folder, file_reg, model_names, time_strs, batch_size, num_classes)
batch_data = next(val_gen)
print(batch_data[0].shape, batch_data[1].shape)
batch_data = next(val_gen)
print(batch_data[0].shape, batch_data[1].shape)

print('*' * 80)
data_dim = batch_data[0].shape[-1]
print('data_dim: %s' % data_dim)

Using TensorFlow backend.


num_classes: 14951
************************************************************
timesteps: 1
len(train_data): 297000
batch_size: 8
steps_per_epoch_train: 37125
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionResNetV2_200_20180312-050926.h5
  200_20180312-050926
(8, 1536) (8, 14951)
(8, 1536) (8, 14951)
************************************************************
len(val_data): 3000
batch_size: 8
steps_per_epoch_val: 376
File existed: /data1/kaggle/landmark-recognition-challenge/feature/feature_InceptionResNetV2_200_20180312-050926.h5
  200_20180312-050926
(8, 1536) (8, 14951)
(8, 1536) (8, 14951)
********************************************************************************
data_dim: 1536
CPU times: user 5.96 s, sys: 20.4 s, total: 26.4 s
Wall time: 26.5 s


In [15]:
%%time
def load_time_str_feature_test(feature_folder, file_reg, model_names, time_str):
    x_test_time_strs = []
    for model_name in model_names:
        file_name = file_reg % (model_name, time_str)
        file_path = os.path.join(feature_folder, file_name)
        x_test_time_str = load_h5_test(feature_folder, file_reg, model_name, time_str)
#         x_test_time_str = np.round(x_test_time_str, decimals=3) # 导致后面的全链接神经网络分类器不收敛
        x_test_time_strs.append(x_test_time_str)
    x_test_time_strs = np.concatenate(x_test_time_strs, axis=-1)
#     print(x_test_time_strs.shape)
    return x_test_time_strs

x_test = load_time_str_feature_test(feature_folder, file_reg, model_names, time_strs[0])
print(x_test.shape)

(115942, 1536)
CPU times: user 396 ms, sys: 1.27 s, total: 1.67 s
Wall time: 1.66 s


In [16]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

len_data = x_train.shape[0]
len_test = x_test.shape[0]
print('len_data: %s' % len_data)
print('len_test: %s' % len_test)

(1219426, 2048)
(1219426,)
(115942, 1536)
len_data: 1219426
len_test: 115942


## Build NN

In [17]:
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

In [18]:
def get_lr(x):
    lr = round(3e-4 * 0.97 ** x, 6)
    if lr < 5e-4:
        lr = 5e-4
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)
callbacks = []
callbacks = [annealer]

log_dir = os.path.join(log_folder, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:/data1/kaggle/landmark-recognition-challenge/log/Google_LandMark_Rec_3. Train-Predict_Mix3model_20180410_123455


In [19]:
model = Sequential()
model.add(Dense(8192, input_shape=x_test.shape[1:]))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(len_unique_landmark_ids, activation='softmax'))

model.compile(
    optimizer=Adam(lr=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              12591104  
_________________________________________________________________
batch_normalization_1 (Batch (None, 8192)              32768     
_________________________________________________________________
dropout_1 (Dropout)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              33558528  
_________________________________________________________________
batch_normalization_2 (Batch (None, 4096)              16384     
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 14951)             61254247  
Total para

In [21]:
# %%time
# hist = model.fit(
#     x=x_train,
#     y=y_train,
#     batch_size=batch_size,
#     epochs=20, #Increase this when not on Kaggle kernel
#     verbose=1,  #1 for ETA, 0 for silent
#     callbacks=callbacks)

In [22]:
%%time
hist = model.fit_generator(
    train_gen,
    steps_per_epoch=steps_per_epoch_train,
    epochs=20, #Increase this when not on Kaggle kernel
    verbose=1,  #1 for ETA, 0 for silent
    callbacks=callbacks,
    max_queue_size=2,
    workers=4,
    use_multiprocessing=False,
    validation_data=val_gen,
    validation_steps=steps_per_epoch_val
)

Epoch 1/20
37125/37125 [==============================] - 5072s 137ms/step - loss: 5.6396 - acc: 0.2443 - val_loss: 4.3697 - val_acc: 0.3647
Epoch 2/20
37125/37125 [==============================] - 5077s 137ms/step - loss: 4.5736 - acc: 0.3318 - val_loss: 4.4138 - val_acc: 0.3787
Epoch 3/20
37125/37125 [==============================] - 5082s 137ms/step - loss: 4.3591 - acc: 0.3544 - val_loss: 4.5057 - val_acc: 0.3833
Epoch 4/20
37125/37125 [==============================] - 5080s 137ms/step - loss: 4.2074 - acc: 0.3672 - val_loss: 4.7812 - val_acc: 0.3700
Epoch 5/20
37125/37125 [==============================] - 5077s 137ms/step - loss: 4.1298 - acc: 0.3727 - val_loss: 4.9044 - val_acc: 0.3707
Epoch 6/20
37125/37125 [==============================] - 5076s 137ms/step - loss: 4.0500 - acc: 0.3786 - val_loss: 5.1547 - val_acc: 0.3680
Epoch 7/20
37125/37125 [==============================] - 5076s 137ms/step - loss: 3.9810 - acc: 0.3833 - val_loss: 5.1469 - val_acc: 0.3617
Epoch 8/20
37

KeyboardInterrupt: 

In [ ]:
final_loss, final_acc = model.evaluate_generator(val_gen, steps=steps_per_epoch_val)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
run_name_acc = run_name + '_' + str(int(final_acc*10000)).zfill(4)

In [ ]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_folder, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, run_name_acc)

## Predict

In [ ]:
y_pred = model.predict(x_test, batch_size=batch_size)
print(y_pred.shape)

In [ ]:
# y_pred_file = os.path.join(model_folder, '%s.npy' % run_name_acc)
# np.save(y_pred_file, y_pred)

# y_pred_reload = np.load(y_pred_file)
# print(y_pred_file)
# print(y_pred_reload.shape)

In [ ]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(cwd, 'input', 'data_test', 'test'))
print(files[:10])

In [ ]:
# 这里证明ImageDataGenerator()得到的图片名称list才是正确
gen = ImageDataGenerator()
image_size = (299, 299)
# batch_size = 128
test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

In [ ]:
%%time
max_indexes = np.argmax(y_pred, -1)
print(max_indexes.shape)

test_dict = {}
for i, paire in enumerate(zip(test_generator.filenames, max_indexes)):
    image_name, indx = paire[0], paire[1]
    image_id = image_name[5:-4]
#     test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])
    test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])

#确认图片的id是否能与ImageDataGenerator()对应上
for key in list(test_dict.keys())[:10]:
    print('%s  %s' % (key, test_dict[key]))

In [ ]:
display(sample_submission_csv.head(2))

In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = sample_submission_csv.iloc[i, 0]
#     landmarks = sample_submission_csv.iloc[i, 1]
    if image_id in test_dict:
        pred_landmarks = test_dict[image_id]
#         print('%s  %s' % (image_id, pred_landmarks))
        sample_submission_csv.iloc[i, 1] = pred_landmarks
    else:
#         print(image_id)
#         sample_submission_csv.iloc[i, 1] = '9633 1.0' # 属于9633的类最多，所以全都设置成这个类，可能会比设置成空得到的结果好
        sample_submission_csv.iloc[i, 1] = '' # 设置成空
    count += 1
    if count % 10000 == 0:
        print(int(count/10000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)
print('Done !')